In [ ]:
import pandas as pd

## Load in the "rosetta stone" file

I made this file using QGIS, the open-source mapping software. I loaded in the US Census 2010 block-level shapefile for DC and the DC police boundaries shapefile [from here](https://opendata.dc.gov/datasets/police-service-areas?geometry=-77.997%2C38.707%2C-76.032%2C39.081). I then used the block centroids, provided by the census, to colect them within each zone. Since the centroids, by nature, are a "half a block" from the nearest street, this is more reliable than a polygon-in-polygon calculation.

I'll write up my steps for that soonest.

In [2]:
rosetta_df = pd.read_csv('../data/washington_dc/rosetta.csv')

In [3]:
rosetta_df

,GEOID10,DISTRICT,PSA,NAME,POLDIST_ID,GIS_ID,SECTOR
0,110010104001019,7,707,707,7,psa_4,7D2
1,110010073011041,7,707,707,7,psa_4,7D2
2,110010098031002,7,707,707,7,psa_4,7D2
3,110010098033004,7,707,707,7,psa_4,7D2
4,110010098071009,7,708,708,7,psa_1,7D3
...,...,...,...,...,...,...,...
6502,110010015004013,2,201,201,2,psa_55,2D1
6503,110010015004005,2,201,201,2,psa_55,2D1
6504,110010015001012,2,201,201,2,psa_55,2D1
6505,110010013012013,2,203,203,2,psa_50,2D1


## Load in the population data

I downloaded the population files from [census.data.gov](https://census.data.gov). Here's the ["productDownload_2020-06-07T194243" zip file](https://s3.amazonaws.com/media.johnkeefe.net/census-by-precinct/productDownload_2020-06-07T194243.zip) I got from the census, which you'll need if you're running this part — along with your paths to the file instead of mine.

In [4]:
# census P3 by block
df_p3 = pd.read_csv('/Volumes/JK_Smarts_Data/precinct_project/DC/productDownload_2020-06-07T194243/DECENNIALSF12010.P3_data_with_overlays_2020-06-07T194221.csv')

In [6]:
df_p3.reset_index(inplace=True)

In [7]:
df_p3.drop(0, inplace=True) #drops second column headers

In [8]:
# census P5 by block
df_p5 = pd.read_csv('/Volumes/JK_Smarts_Data/precinct_project/DC/productDownload_2020-06-07T194243/DECENNIALSF12010.P5_data_with_overlays_2020-06-07T194221.csv')

In [9]:
df_p5.reset_index(inplace=True)
df_p5.drop(0, inplace=True)

In [10]:
df_p3.shape, df_p5.shape

((6507, 11), (6507, 20))

In [11]:
main_df = df_p3.merge(df_p5, on='GEO_ID')

In [12]:
main_df.shape

(6507, 30)

In [13]:
main_df

,index_x,GEO_ID,NAME_x,P003001,P003002,P003003,P003004,P003005,P003006,P003007,...,P005008,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017
0,1,1000000US110010040021000,"Block 1000, Block Group 1, Census Tract 40.02,...",273,199,24,0,10,1,22,...,0,11,57,17,12,0,0,0,22,6
1,2,1000000US110010040021001,"Block 1001, Block Group 1, Census Tract 40.02,...",407,346,27,1,17,0,9,...,1,7,45,33,4,0,0,0,8,0
2,3,1000000US110010040022000,"Block 2000, Block Group 2, Census Tract 40.02,...",464,377,39,0,30,0,9,...,0,8,34,22,0,0,2,0,9,1
3,4,1000000US110010040022001,"Block 2001, Block Group 2, Census Tract 40.02,...",345,298,26,0,7,0,2,...,0,11,20,16,1,0,0,0,2,1
4,5,1000000US110010040022003,"Block 2003, Block Group 2, Census Tract 40.02,...",315,239,24,1,35,0,9,...,0,7,27,18,0,0,0,0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6502,6503,1000000US110010102001005,"Block 1005, Block Group 1, Census Tract 102, D...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6503,6504,1000000US110010106003004,"Block 3004, Block Group 3, Census Tract 106, D...",130,46,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6504,6505,1000000US110010089032001,"Block 2001, Block Group 2, Census Tract 89.03,...",193,10,175,0,0,0,0,...,0,8,3,2,1,0,0,0,0,0
6505,6506,1000000US110010095052004,"Block 2004, Block Group 2, Census Tract 95.05,...",52,2,49,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0


In [14]:
main_df.columns

Index(['index_x', 'GEO_ID', 'NAME_x', 'P003001', 'P003002', 'P003003',
       'P003004', 'P003005', 'P003006', 'P003007', 'P003008', 'index_y',
       'NAME_y', 'P005001', 'P005002', 'P005003', 'P005004', 'P005005',
       'P005006', 'P005007', 'P005008', 'P005009', 'P005010', 'P005011',
       'P005012', 'P005013', 'P005014', 'P005015', 'P005016', 'P005017'],
      dtype='object')

In [15]:
main_df['GEOID10'] = main_df['GEO_ID'].str[9:].astype(int)

In [17]:
main_df.drop(columns=['index_x'], inplace = True)

In [22]:
main_df.drop(columns=['index_y', 'NAME_y'], inplace = True)

In [23]:
main_df.columns

Index(['GEO_ID', 'NAME_x', 'P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008', 'P005001', 'P005002',
       'P005003', 'P005004', 'P005005', 'P005006', 'P005007', 'P005008',
       'P005009', 'P005010', 'P005011', 'P005012', 'P005013', 'P005014',
       'P005015', 'P005016', 'P005017', 'GEOID10'],
      dtype='object')

In [18]:
rosetta_df.shape

(6507, 7)

In [20]:
rosetta_df.dtypes

GEOID10        int64
DISTRICT       int64
PSA            int64
NAME           int64
POLDIST_ID     int64
GIS_ID        object
SECTOR        object
dtype: object

In [19]:
main_df.dtypes

GEO_ID     object
NAME_x     object
P003001    object
P003002    object
P003003    object
P003004    object
P003005    object
P003006    object
P003007    object
P003008    object
index_y     int64
NAME_y     object
P005001    object
P005002    object
P005003    object
P005004    object
P005005    object
P005006    object
P005007    object
P005008    object
P005009    object
P005010    object
P005011    object
P005012    object
P005013    object
P005014    object
P005015    object
P005016    object
P005017    object
GEOID10     int64
dtype: object

In [25]:
# need to make all those columns numeric
main_df[['P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008', 'P005001', 'P005002',
       'P005003', 'P005004', 'P005005', 'P005006', 'P005007', 'P005008',
       'P005009', 'P005010', 'P005011', 'P005012', 'P005013', 'P005014',
       'P005015', 'P005016', 'P005017']] = main_df[['P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008', 'P005001', 'P005002',
       'P005003', 'P005004', 'P005005', 'P005006', 'P005007', 'P005008',
       'P005009', 'P005010', 'P005011', 'P005012', 'P005013', 'P005014',
       'P005015', 'P005016', 'P005017']].apply(pd.to_numeric)

In [26]:
## Add districts to every block's demo data
block_data = main_df.merge(rosetta_df, on="GEOID10", how="left")

In [27]:
block_data.shape

(6507, 34)

In [28]:
block_data

,GEO_ID,NAME_x,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,...,P005015,P005016,P005017,GEOID10,DISTRICT,PSA,NAME,POLDIST_ID,GIS_ID,SECTOR
0,1000000US110010040021000,"Block 1000, Block Group 1, Census Tract 40.02,...",273,199,24,0,10,1,22,17,...,0,22,6,110010040021000,3,303,303,3,psa_37,3D3
1,1000000US110010040021001,"Block 1001, Block Group 1, Census Tract 40.02,...",407,346,27,1,17,0,9,7,...,0,8,0,110010040021001,3,303,303,3,psa_37,3D3
2,1000000US110010040022000,"Block 2000, Block Group 2, Census Tract 40.02,...",464,377,39,0,30,0,9,9,...,0,9,1,110010040022000,3,303,303,3,psa_37,3D3
3,1000000US110010040022001,"Block 2001, Block Group 2, Census Tract 40.02,...",345,298,26,0,7,0,2,12,...,0,2,1,110010040022001,3,303,303,3,psa_37,3D3
4,1000000US110010040022003,"Block 2003, Block Group 2, Census Tract 40.02,...",315,239,24,1,35,0,9,7,...,0,9,0,110010040022003,3,303,303,3,psa_37,3D3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6502,1000000US110010102001005,"Block 1005, Block Group 1, Census Tract 102, D...",0,0,0,0,0,0,0,0,...,0,0,0,110010102001005,1,103,103,1,psa_12,1D1
6503,1000000US110010106003004,"Block 3004, Block Group 3, Census Tract 106, D...",130,46,84,0,0,0,0,0,...,0,0,0,110010106003004,5,501,501,5,psa_35,5D1
6504,1000000US110010089032001,"Block 2001, Block Group 2, Census Tract 89.03,...",193,10,175,0,0,0,0,8,...,0,0,0,110010089032001,5,507,507,5,psa_23,5D3
6505,1000000US110010095052004,"Block 2004, Block Group 2, Census Tract 95.05,...",52,2,49,0,0,0,0,1,...,0,0,0,110010095052004,4,406,406,4,psa_54,4D2


In [29]:
block_data.to_csv('./temp_data/dc_2010blocks_2020policedistricts_population.csv', index=False)

In [30]:
## Check for duplicates
block_data.duplicated(subset='GEOID10', keep='first').sum()

0

In [31]:
import numpy as np
pivot = pd.pivot_table(block_data, index="DISTRICT", aggfunc=np.sum)

In [32]:
pivot

,GEOID10,NAME,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,...,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017,POLDIST_ID,PSA
DISTRICT,,,,,,,,,,,,,,,,,,,,,
1,113200366788363744,107915,70188,36738,26788,282,3364,40,980,1996,...,3493,1828,402,43,36,1,816,367,1029,107915
2,186026955156884448,346078,140138,115011,8013,294,10393,71,2125,4231,...,10208,7402,254,114,85,22,1686,645,3382,346078
3,56325140613156680,156028,95326,48393,30854,432,4570,67,7410,3600,...,15386,5959,803,219,55,27,7114,1209,1536,156028
4,117490722860241392,431694,100226,20443,62398,475,1695,73,11391,3751,...,19677,5310,1198,253,34,32,11080,1770,4272,431694
5,81407470139746432,372868,55839,7106,45006,236,648,18,1417,1408,...,3120,898,482,77,18,2,1321,322,3700,372868
6,93728591642625552,515051,74065,1284,70447,222,147,13,714,1238,...,1624,279,489,30,0,2,639,185,5112,515051
7,67656202377240896,433643,65941,2496,61619,138,239,20,337,1092,...,1241,331,444,21,10,0,267,168,4305,433643


In [33]:
pivot.reset_index(inplace=True)

In [34]:
# drop columns whose sums make no sense
pivot.drop(columns=['GEOID10','POLDIST_ID','PSA'], inplace=True)

In [36]:
pivot.drop(columns=['NAME'], inplace=True)

In [37]:
pivot.columns

Index(['DISTRICT', 'P003001', 'P003002', 'P003003', 'P003004', 'P003005',
       'P003006', 'P003007', 'P003008', 'P005001', 'P005002', 'P005003',
       'P005004', 'P005005', 'P005006', 'P005007', 'P005008', 'P005009',
       'P005010', 'P005011', 'P005012', 'P005013', 'P005014', 'P005015',
       'P005016', 'P005017'],
      dtype='object')

In [38]:
pivot

,DISTRICT,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,P005001,...,P005008,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017
0,1,70188,36738,26788,282,3364,40,980,1996,70188,...,164,1629,3493,1828,402,43,36,1,816,367
1,2,140138,115011,8013,294,10393,71,2125,4231,140138,...,439,3586,10208,7402,254,114,85,22,1686,645
2,3,95326,48393,30854,432,4570,67,7410,3600,95326,...,296,2391,15386,5959,803,219,55,27,7114,1209
3,4,100226,20443,62398,475,1695,73,11391,3751,100226,...,311,1981,19677,5310,1198,253,34,32,11080,1770
4,5,55839,7106,45006,236,648,18,1417,1408,55839,...,96,1086,3120,898,482,77,18,2,1321,322
5,6,74065,1284,70447,222,147,13,714,1238,74065,...,75,1053,1624,279,489,30,0,2,639,185
6,7,65941,2496,61619,138,239,20,337,1092,65941,...,70,924,1241,331,444,21,10,0,267,168


In [40]:
pivot.to_csv('../data/washington_dc/dc_2010pop_by_2020police_districts.csv', index=False)

In [41]:
pivot['P003001'].sum()

601723

In [43]:
pivot_psa = pd.pivot_table(block_data, index="PSA", aggfunc=np.sum)

In [50]:
pivot_psa.reset_index(inplace=True)

In [51]:
pivot_psa.columns

Index(['PSA', 'DISTRICT', 'GEOID10', 'NAME', 'P003001', 'P003002', 'P003003',
       'P003004', 'P003005', 'P003006', 'P003007', 'P003008', 'P005001',
       'P005002', 'P005003', 'P005004', 'P005005', 'P005006', 'P005007',
       'P005008', 'P005009', 'P005010', 'P005011', 'P005012', 'P005013',
       'P005014', 'P005015', 'P005016', 'P005017', 'POLDIST_ID'],
      dtype='object')

In [52]:
pivot_psa

,PSA,DISTRICT,GEOID10,NAME,P003001,P003002,P003003,P003004,P003005,P003006,...,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017,POLDIST_ID
0,101,64,7040643468491659,6464,6407,3259,1469,18,1331,4,...,134,421,202,31,1,3,0,141,43,64
1,102,132,14521328766612804,13464,4864,1275,3218,14,200,3,...,92,172,79,30,5,4,0,36,18,132
2,103,164,18041654668384396,16892,5972,2915,2348,25,366,3,...,182,359,193,40,5,4,0,66,51,164
3,104,79,8690796693785585,8216,8890,4825,3421,21,230,11,...,233,415,214,61,2,10,0,94,34,79
4,105,75,8250756085154956,7875,5620,1096,4030,51,161,3,...,149,239,64,58,5,0,0,89,23,75
5,106,237,26072386606591036,25122,11181,6968,3349,40,388,7,...,241,589,385,39,4,3,0,90,68,237
6,107,142,15621429615360044,15194,15433,7946,6462,67,341,4,...,322,804,414,122,10,4,1,168,85,142
7,108,136,14961370883983136,14688,11821,8454,2491,46,347,5,...,276,494,277,21,11,8,0,132,45,136
8,201,402,22112012823552976,40401,14205,11678,1274,18,637,6,...,334,897,657,14,7,5,2,144,68,402
9,202,470,25852352488216800,47470,11906,10077,534,34,683,7,...,381,812,636,4,17,7,1,99,48,470


In [55]:
# drop columns whose sums make no sense
pivot_psa.drop(columns=['DISTRICT', 'GEOID10', 'NAME', 'POLDIST_ID'], inplace=True)

In [56]:
pivot_psa

,PSA,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,P005001,...,P005008,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017
0,101,6407,3259,1469,18,1331,4,149,177,6407,...,8,134,421,202,31,1,3,0,141,43
1,102,4864,1275,3218,14,200,3,44,110,4864,...,8,92,172,79,30,5,4,0,36,18
2,103,5972,2915,2348,25,366,3,82,233,5972,...,16,182,359,193,40,5,4,0,66,51
3,104,8890,4825,3421,21,230,11,115,267,8890,...,21,233,415,214,61,2,10,0,94,34
4,105,5620,1096,4030,51,161,3,107,172,5620,...,18,149,239,64,58,5,0,0,89,23
5,106,11181,6968,3349,40,388,7,120,309,11181,...,30,241,589,385,39,4,3,0,90,68
6,107,15433,7946,6462,67,341,4,206,407,15433,...,38,322,804,414,122,10,4,1,168,85
7,108,11821,8454,2491,46,347,5,157,321,11821,...,25,276,494,277,21,11,8,0,132,45
8,201,14205,11678,1274,18,637,6,190,402,14205,...,46,334,897,657,14,7,5,2,144,68
9,202,11906,10077,534,34,683,7,142,429,11906,...,43,381,812,636,4,17,7,1,99,48


In [57]:
pivot_psa.to_csv('../data/washington_dc/dc_2010pop_by_2020public_service_area.csv', index=False)

Done!